In [19]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
device = torch.device('cuda')

In [20]:
preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


class subimages_dataset(Dataset):
    def __init__(self, df_path, train = False):
        self.df = pd.read_csv(df_path)
        self.train = train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        subimages_path = '/scratch/cz2064/myjupyter/BDML/Project/Data/data_subimages/'
        file_id = self.df.iloc[idx]['File ID']
        file_name = self.df.iloc[idx]['File Name']
        subimages_file = self.df.iloc[idx]['Subimage File']
        subimages_path = subimages_path + file_id + '/' + subimages_file
        image = Image.open(subimages_path)
        image_tensor = preprocess(image)
        label = self.df.iloc[idx]['label']
        sample = {'x': image_tensor, 'y': label}
        
        return sample   

In [21]:
train_df_path = '/scratch/cz2064/myjupyter/BDML/Project/jupyter_phase2/Train_Val_Test/train_subimages.csv'
val_df_path = '/scratch/cz2064/myjupyter/BDML/Project/jupyter_phase2/Train_Val_Test/val_subimages.csv'
test_df_path = '/scratch/cz2064/myjupyter/BDML/Project/jupyter_phase2/Train_Val_Test/test_subimages.csv'

BATCH_SIZE = 8
train_loader = DataLoader(subimages_dataset(train_df_path), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(subimages_dataset(val_df_path), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(subimages_dataset(test_df_path), batch_size=BATCH_SIZE, shuffle=True)

In [22]:
sample = next(iter(train_loader))
sample_input = sample['x'].to(device,dtype=torch.float)
sample_output = sample['y']

AssertionError: 
The NVIDIA driver on your system is too old (found version 10010).
Please update your GPU driver by downloading and installing a new
version from the URL: http://www.nvidia.com/Download/index.aspx
Alternatively, go to: https://pytorch.org to install
a PyTorch version that has been compiled with your version
of the CUDA driver.

In [14]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8 MB 19 kB/s s eta 0:00:01   |▉                               | 17.4 MB 7.7 MB/s eta 0:01:29     |█▋                              | 35.2 MB 7.7 MB/s eta 0:01:27     |███                             | 64.9 MB 46.6 MB/s eta 0:00:14     |███                             | 67.5 MB 46.6 MB/s eta 0:00:14     |███▍                            | 75.0 MB 46.6 MB/s eta 0:00:14     |█████                           | 107.7 MB 52.1 MB/s eta 0:00:12     |█████▍                          | 117.6 MB 52.1 MB/s eta 0:00:12     |█████▉                          | 127.6 MB 52.1 MB/s eta 0:00:12     |██████                          | 130.1 MB 52.1 MB/s eta 0:00:12     |███████                         | 155.2 MB 18.5 MB/s eta 0:00:30     |███████▏                        | 157.8 MB 18.5 MB/s eta 0:00:30     |███████▎                        | 160.3 MB 18.5 MB/s eta 0:00:30     |████████                

In [ ]:
VGG_11 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)

In [ ]:
class VGG_CAM(nn.Module):
    def __init__(self, features = VGG_11.features, n_classes = 3):
        super(VGG_CAM, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512, n_classes)

    def forward(self, x):
        x = self.features(x)
        self.featuremap1 = x.detach()
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model = VGG_CAM().to(device)

In [ ]:
model(sample_input)

In [ ]:
def train(model, train_loader=train_loader, val_loader=val_loader, learning_rate=0.0001, num_epoch=10):
    start_time = time.time()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)
    train_loss_return = []
    train_acc_return = []
    val_loss_return = []
    val_acc_return = []
    best_acc = 0
    
    for epoch in range(num_epoch):
        # Training steps
        correct = 0
        total = 0
        predictions = []
        truths = []
        model.train()
        train_loss_list = []
        for i, (sample) in enumerate(train_loader):
            data = sample['x'].to(device)
            labels = sample['y'].to(device)
            outputs = model(data)
            pred = outputs.data.max(-1)[1]
            predictions += list(pred.cpu().numpy())
            truths += list(labels.cpu().numpy())
            total += labels.size(0)
            correct += (pred == labels).sum()
            model.zero_grad()
            loss = loss_fn(outputs, labels)
            train_loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
        # report performance
        acc = (100 * correct / total)
        train_acc_return.append(acc)
        train_loss_every_epoch = np.average(train_loss_list)
        train_loss_return.append(train_loss_every_epoch)
        print('----------Epoch{:2d}/{:2d}----------'.format(epoch+1,num_epoch))
        print('Train set | Loss: {:6.4f} | Accuracy: {:4.2f}% '.format(train_loss_every_epoch, acc))
        
        # Evaluate after every epochh
        correct = 0
        total = 0
        model.eval()
        predictions = []
        truths = []
        val_loss_list = []
        with torch.no_grad():
            for i, (sample) in enumerate(val_loader):
                data = sample['x'].to(device)
                labels = sample['y'].to(device)
                outputs = model(data)
                loss = loss_fn(outputs, labels)
                val_loss_list.append(loss.item())
                pred = outputs.data.max(-1)[1]
                predictions += list(pred.cpu().numpy())
                truths += list(labels.cpu().numpy())
                total += labels.size(0)
                correct += (pred == labels).sum()
            # report performance
            acc = (100 * correct / total)
            val_acc_return.append(acc)
            val_loss_every_epoch = np.average(val_loss_list)
            val_loss_return.append(val_loss_every_epoch)
            if acc > best_acc:
                best_acc = acc
                best_model_wts = model.state_dict()
            save_model(model,train_loss_return,train_acc_return,val_loss_return,val_acc_return,best_model_wts)
            elapse = time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))
            print('Test set | Loss: {:6.4f} | Accuracy: {:4.2f}% | time elapse: {:>9}'\
                  .format(val_loss_every_epoch, acc,elapse))
    return model,train_loss_return,train_acc_return,val_loss_return,val_acc_return,best_model_wts

def save_model(model,train_loss_return,train_acc_return,val_loss_return,val_acc_return,best_model_wts):
    state = {'best_model_wts':best_model_wts, 'model':model, \
             'train_loss':train_loss_return, 'train_acc':train_acc_return,\
             'val_loss':val_loss_return, 'val_acc':val_acc_return}
    torch.save(state, 'checkpoint_Analysis2_VGG_CAM.pt')
    return None

In [ ]:
train(model, train_loader=train_loader, val_loader=val_loader, learning_rate=0.0001, num_epoch=10)